In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

- Reference code/ notebook: https://medium.datadriveninvestor.com/k-fold-and-dropout-in-artificial-neural-network-ea054a89fb4e
- https://machinelearningmastery.com/use-keras-deep-learning-models-scikit-learn-python/
- You can also check my other notebook based on neural network [here](https://www.kaggle.com/nitishkumar3099/neural-network-feature-selection-tps-feb-2022/edit/run/88711391).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns




from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from scipy.stats import mode


from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
 
from matplotlib import ticker
import time
import warnings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:f}'.format)
warnings.filterwarnings('ignore')


RANDOM_STATE = 12 
FOLDS = 5

In [ ]:
data = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/test.csv')
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv')



In [ ]:
display(data.head())
display(test.head())

In [ ]:
le = LabelEncoder()
data["target"] = le.fit_transform(data["target"])
target = le.fit_transform(data.target)

display(data.head())
display(data.tail())

In [ ]:
#from keras.utils.np_utils import to_categorical
#output_category = to_categorical(y, num_classes=None)
#output_category

In [ ]:
useful_cols = [c for c in data.columns]

In [ ]:
useful_cols.remove('target')
useful_cols.remove('row_id')

In [ ]:

autoscaler = StandardScaler()
data[useful_cols] = autoscaler.fit_transform(data[useful_cols])

In [ ]:
data.head()

In [ ]:
cols = [c for c in data.columns]


In [ ]:
cols.remove('row_id')
cols.remove('target')

In [ ]:
X = data[cols]

In [ ]:
X.head()

In [ ]:
output_category = data['target']

In [ ]:
output_category.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, output_category, test_size=0.25)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
#builing the neural net
from keras import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
# K fold validation
def build_classifier():
    model=Sequential()
    # first layer 
    # for initializing the weight we use he_normal
    model.add(Dense(units=512,kernel_initializer='he_normal',activation='relu',input_dim=286))
    #2nd layer
    model.add(Dense(units=256,kernel_initializer='he_normal',kernel_regularizer = regularizers.l2(0.001),activation='relu'))
    #3rd layer
    model.add(Dense(units=64,kernel_initializer='he_normal',activation='relu'))
    model.add(Dense(units=512,kernel_initializer='he_normal',kernel_regularizer = regularizers.l1_l2(l1=0.001, l2=0.002),activation='relu'))
    # to avoid overfitting we add dropout and kernel_regularizer
    model.add(layers.Dropout(0.5))
    model.add(Dense(units=128,kernel_initializer='he_normal',activation='relu'))
    model.add(Dense(units = 64, kernel_initializer='he_normal',activation = 'relu'))
    model.add(Dense(units=512,kernel_initializer='he_normal',kernel_regularizer = regularizers.l1_l2(l1=0.001, l2=0.002),activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(Dense(units=128,kernel_initializer='he_normal',activation='relu'))
    model.add(Dense(units = 64, activation = 'relu'))
    model.add(Dense(10, kernel_initializer='random_uniform',activation = 'softmax'))
    # compiling


    model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [ ]:
classifier = KerasClassifier(build_fn=build_classifier, batch_size=512, epochs=100, verbose=0)

In [ ]:
seed  = 17171
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

In [ ]:
#accuracies =  cross_val_score(estimator=classifier, X= X, y=output_category,cv=5, n_jobs=-1)
#accuracies

In [ ]:
results = cross_val_score(classifier, X, output_category, cv=kfold)


In [ ]:
print(results.mean())